<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [0]:
!pip install ipython-autotime

%load_ext autotime

In [0]:
class Graph:
  def __init__(self, n):
    self.n = n # number of nodes
    self.w = [][] # matrix of distances w(u, v) 

In [33]:
lines = open("burma14.tsp", "r").readlines()
print(lines)
n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
t = lines[4].split()[1]
coordinates = lines[9]

'''for line in lines:
  if line.split()[0] == 'DIMENSION:':
    n = line.split()[0]
  else if line.split()[0] == 'EDGE_WEIGHT_TYPE:'
    t = line.split()[0]
  print()'''

V = [] # array of vertexes/coordinates
for i in range(9, 9 + n - 1):
  V.append([lines[i].split()[1], lines[i].split()[2]])

print(V)

['NAME: burma14\n', 'TYPE: TSP\n', 'COMMENT: 14-Staedte in Burma (Zaw Win)\n', 'DIMENSION: 14\n', 'EDGE_WEIGHT_TYPE: GEO\n', 'EDGE_WEIGHT_FORMAT: FUNCTION \n', 'DISPLAY_DATA_TYPE: COORD_DISPLAY\n', 'NODE_COORD_SECTION\n', '   1  16.47       96.10\n', '   2  16.47       94.44\n', '   3  20.09       92.54\n', '   4  22.39       93.37\n', '   5  25.23       97.24\n', '   6  22.00       96.05\n', '   7  20.47       97.02\n', '   8  17.20       96.29\n', '   9  16.30       97.38\n', '  10  14.05       98.12\n', '  11  16.53       97.38\n', '  12  21.52       95.59\n', '  13  19.41       97.13\n', '  14  20.09       94.55\n', 'EOF\n', '\n', '\n', '\n']
[['16.47', '94.44'], ['20.09', '92.54'], ['22.39', '93.37'], ['25.23', '97.24'], ['22.00', '96.05'], ['20.47', '97.02'], ['17.20', '96.29'], ['16.30', '97.38'], ['14.05', '98.12'], ['16.53', '97.38'], ['21.52', '95.59'], ['19.41', '97.13'], ['20.09', '94.55']]


In [0]:
# global values
d = [][]
phi = [][]



In [0]:
import math
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == "EUC_2D":
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    RRR = 6378.388
    q1 = math.cos(u[1] - v[1])
    q2 = math.cos(u[0] - v[0])
    q3 = math.cos(u[0] + v[0])
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

In [0]:
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  if S = {v}: # Caso base: la soluzione è il peso dell’arco {v,0}
    return w[v, 0]
  else if d[v, S] != NULL: # Distanza già calcolata, ritorna il valore memorizzato
    return d[v,S] 
  else  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    mindist = ∞
    minprec = NULL
    for u in S \ {v}:
      dist = HK_VISIT(u, S \ {v})
      if (dist + weight(u, v)) < mindist:
        mindist = dist + weight(u, v)
        minprec = u
    d[v, S] = mindist
    phi[v, S] = minprec
    return mindist

In [0]:
def HK_TSP (G):
  G = (V, E) # build graph
  d = 0 # initialize matrix d
  phi = 0 # initialize matrix phi
  return HK_VISIT(0, V)

